# Import libraries

In [1]:
import pypots
import os
import sys
from pypots.nn.functional import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012 as preprocess_physionet2012
from missingData.toolkits import toolkits
import math
import scipy.stats as st

2025-04-13 22:35:30.687115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744594530.708896  343382 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744594530.716425  343382 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-13 22:35:30.737444: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/pypots/nn/functional/cuda.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Loading database

### Standard Scaler 

In [2]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-04-13 22:35:50 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-04-13 22:35:50 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-04-13 22:35:50 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-04-13 22:35:50 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-04-13 22:35:50 [INFO]: Loaded successfully!
2025-04-13 22:36:07 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-04-13 22:36:07 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-04-13 22:36:07 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [4]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

# Creating indicating mask and get test_X_ori

### Standard Scaler

In [5]:
dataset_for_testing_ori_standard_norm = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_norm = toolkits.dict_to_list(dataset_for_testing_standard)
indicating_mask_variable_standard_norm, test_X_ori_variable_standard_norm = toolkits.components_mae(dataset_for_testing_ori_standard_norm, dataset_for_testing_standard_norm)
indicating_mask_variable_standard_norm = toolkits.pre_reshape(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.pre_reshape(test_X_ori_variable_standard_norm)
indicating_mask_variable_standard_norm = toolkits.reshape_variable(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.reshape_variable(test_X_ori_variable_standard_norm)

# Inicialize the models

### SAITS

<h4> Inicialize model (Standard)</h4>

In [6]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-13 22:36:31 [INFO]: No given device, using default device: cpu
2025-04-13 22:36:31 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-04-13 22:36:31 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


### BRITS

<h4> Inicialize model (Standard)</h4>

In [ ]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-04 20:21:23 [INFO]: No given device, using default device: cpu
2025-04-04 20:21:23 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250404_T202123
2025-04-04 20:21:23 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250404_T202123/tensorboard
2025-04-04 20:21:23 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


# Train/Load the models

### SAITS

<h4> Load - Standard Scaler </h4>

In [7]:
saits_standard.load("tutorial_results/imputation/saits/standard/SAITS.pypots")

2025-04-13 22:36:35 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/standard/SAITS.pypots


### BRITS

In [44]:
brits_standard.load("tutorial_results/imputation/brits/standard/BRITS.pypots")

2025-03-27 00:20:08 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/standard/BRITS.pypots


# Testing stage

### SAITS

#### Standard Scaler

In [8]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits_standard)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

### BRITS

#### Standard Scaler

In [45]:
brits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, brits_standard)
brits_imputation_standard = toolkits.pre_reshape(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.desnormalization(brits_imputation_standard, scaler_standard)
brits_imputation_standard = toolkits.reshape_variable(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.reshape_variable(brits_imputation_standard_ori)

# Calculate mean absolute error

### SAITS

#### Standard scaler (C/Normalização)

In [9]:
testing_mae_saits_variables_standard, testing_ae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

#### Taking the MAE difference for SAITS model

In [10]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

In [12]:
diff_saits = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 1, 2, variables)
diff_saits

[('Cholesterol', 0.5328215208430203),
 ('SaO2', 0.18297989172603718),
 ('TroponinI', 0.1272326176203385),
 ('TroponinT', 0.10384946406267459),
 ('Lactate', 0.07237066715154516)]

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [13]:
indices = [6, 16, 28, 31, 32]

results_bootstrap_saits = []

for i in indices:
       results_bootstrap_saits.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 1, i, 4))

### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps = [] 
standards_deviations = []
for i in range(len(results_bootstrap_saits)):
    means_bootstraps.append(np.mean(results_bootstrap_saits[i]))
    standards_deviations.append(np.std(results_bootstrap_saits[i]))


### Calculating lower bound and upper bound

In [39]:
lower_bounds = []
upper_bounds = []
for i in range(len(results_bootstrap_saits)):
    lower_bounds.append(means_bootstraps[i] - st.norm.ppf(1-0.05/2) * standards_deviations[i])
    upper_bounds.append(means_bootstraps[i] + st.norm.ppf(1-0.05/2) * standards_deviations[i])

### Mean values of lower bound and upper bound


In [44]:
mean_values_ci = []
for i in range(len(lower_bounds)):
    mean_values_ci.append((lower_bounds[i] + upper_bounds[i]) / 2)

### BRITS

#### Standard scaler (C/Normalização)

In [11]:
testing_mae_brits_variables_standard = toolkits.calculate_mae(brits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

NameError: name 'brits_imputation_standard' is not defined

### Taking the MAE difference for BRITS model

In [ ]:
diff_brits = []

for i in range(len(testing_mae_brits_variables_standard[1])):
    
    diff_brits.append(math.fabs(testing_mae_brits_variables_standard[1][i] - testing_mae_brits_variables_standard[2][i]))    


In [ ]:
diff_brits.sort(reverse=True)

In [ ]:
diff_top_5_brits = diff_brits[0:5]
diff_top_5_brits

### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

# Results mean absolute error

In [17]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

#### SAITS

<h4>Standard Scaler (C/Normalização)</h4>

In [13]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.35549184664706085
ALT : 0.34616397235246166
AST : 0.283203929613111
Albumin : 0.5865378684716107
BUN : 0.23549456829210566
Bilirubin : 0.29213390018033225
Cholesterol : 0.8619142944054452
Creatinine : 0.2272360108358054
DiasABP : 0.25696352027850733
FiO2 : 0.4843457493583031
GCS : 0.27782594851598286
Glucose : 0.5574566141755575
HCO3 : 0.42235495223366226
HCT : 0.5478476366595461
HR : 0.2786515469045242
K : 0.6173738181988982
Lactate : 0.3602081488093162
MAP : 0.276148040474521
MechVent : 0.016131509283529688
Mg : 0.5428868319523208
NIDiasABP : 0.23805346595387422
NIMAP : 0.17731126967659114
NISysABP : 0.3059993623684565
Na : 0.5094466185029012
PaCO2 : 0.5448751678244528
PaO2 : 0.5044175503727509
Platelets : 0.37910753219049526
RespRate : 0.4975549864022565
SaO2 : 0.395203639666887
SysABP : 0.2876315375696396
Temp : 0.3654739421516718
TroponinI : 0.5742820858879293
TroponinT : 0.562502744207675
Urine : 0.4555801478350003
WBC : 0.15

#### BRITS

<h4>Standard Scaler (C/Normalização)</h4>

In [47]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard, subgroups, variables)

BRITS - MAE
************
General
-------------
ALP : 0.4591248631105258
ALT : 0.2098298479270722
AST : 0.27798859929904113
Albumin : 0.6778925772529109
BUN : 0.39521960833661646
Bilirubin : 0.3833806392125974
Cholesterol : 0.9463970561439892
Creatinine : 0.3444718062995419
DiasABP : 0.20806233248448402
FiO2 : 0.34404522156046397
GCS : 0.21322463397612945
Glucose : 0.5644091100017328
HCO3 : 0.5047606820899914
HCT : 0.6338702223060172
HR : 0.2431173715923441
K : 0.6534995938478387
Lactate : 0.4167839664888235
MAP : 0.1564062923159312
MechVent : 0.007398844359593802
Mg : 0.5955810654854342
NIDiasABP : 0.15725503986364742
NIMAP : 0.09992748699932849
NISysABP : 0.2543230216923866
Na : 0.645442078485974
PaCO2 : 1.1895743136362507
PaO2 : 0.4776378612780617
Platelets : 0.6445916196510731
RespRate : 0.475295251812804
SaO2 : 0.348066606252948
SysABP : 0.25398352012962805
Temp : 0.240561647460938
TroponinI : 0.667214116433886
TroponinT : 0.29949287221387894
Urine : 0.3333883323992429
WBC : 0.1101